# 15. 콘텍스트 관리자와 else 블록

## 15.1. if문 외에서 else

else는 if뿐만 아니라 for, while, try에서도 사용할 수 있음.

for/else : for 완전히 실행된 후에 else 블록 실행
while/else : 조건식이 거짓이 되어 while 루프를 빠져나온 후에 else
try/else : try에서 예외가 발생하지 않을 때만 else 실행, else의 예외는 else 앞의 except 블록에서 처리되지 않음.

EAFP : Easier to Ask for Forgiveness than Permission, 잘못 된 경우 예외 처리하는 편이 나음.
LBYL : Leap Before You Leap, 호출이나 조회 전 전제 조건 검사, if문 많이 사용

## 15.2. 콘텍스트 관리자와 with 블록

콘텍스트 관리자 객체는 with문을 제어하기 위해 존재

with : try/finally을 단순화하기 위해 설계됨.


exc_type : ZeroDivisionError 등의 예외 클래스
exc_value : 예외 객체, 예외 메시지 등 exception() 생성자에 전달된 인수는 exc_value.args 속성을 이용해 볼 수 있음
traceback : traceback 객체

## 15.3. contextlib 유틸리티

closing() : close() 메서드는 제공하지만 __enter__()/__exit__() 프로토콜을 구현하지 않는 객체로부터 콘텍스트 관리자를 생성하는 함수
suppress : 지정한 예외를 임시로 무시하는 콘텍스트 관리자
@contextmanager : 클래스를 생성하고 프로토콜을 구현하는 대신, 간단한 제너레이터 함수로부터 콘텍스트 관리자를 생성할 수 있게 해주는 데커레이터
ContextDecoratot : 콘텍스트 관리자를 함수 데커레이터로도 사용할 수 있게 해주는 기반 클래스 
ExitStack : 여러 콘텍스트 관리자를 입력할 수 있게 해주는 콘텍스트 관리자. with 블록이 끝나면 ExitStack은 누적된 콘텍스트 관리자들의 __exit__() 메서드를 LIFO 순서로 호출.

## 15.4. @ contextmanager 사용하기

@contextmanager 데커레이터는 콘텍스트 관리자 생성시 작성하는 코드를 줄여 줌. __enter__(), __exit__()  가진 클래스 작성 대신 __enter__() 메서드가 반환할 것을 생성하는 yield 하나를 가진 제너레이터만 구현하면 됨.

yield는 함수 본체 부분을 두 부분으로 나눔.
yield 앞 : with 블록 앞에서 인터프리터가 __enter__()를 호출할 때 실행
yield 뒤 : __exit__() 호출될 때 실행

# 16. 코루틴

## 16.1. 제너레이터 >> 코루틴

제너레이터의 호출자는 send()를 이용해 제너레이터 함수 내부의 yield 표현식의 값이 될 데이터를 전송할 수 있음. 제너레이터 >> 코루틴(호출자에 데이터 생성, 호출자로부터 데이터 받으며 호출자와 협업하는 프로시저)

throw() : 제너레이터 내부에서 처리할 예외를 호출자가 발생시킬 수 있게 해줌
close() : 제너레이터가 종료되도록 만듦

## 16.2. 코루틴으로 사용되는 제너레이터의 기본 동작

코루틴이 가지는 상태 : inspect.getgeneratorstate() 이용해 알 수 있음
 'GEN_CREATED' : 실행을 시작하기 위해 대기하는 상태
 'GEN_RUNNING' : 현재 인터프리터가 실행되는 상태
 'GEN_SUSPENDED' : 현재 yield에서 대기하는 상태
 'GEN_CLOSED' : 실행 완료 상태
 
## 16.3. EX) 이동 평균을 계산하는 코루틴

## 16.4. 코루틴을 기동하기 위한 데커레이터

my_coro.send(x) 첫 호출 전 next(my_coro)를 호출해야.

@coroutine 데커레이터 : 코루틴 편리하게 사용할 수 있도록 기동하는 데커레이터 중 하나

@tornado.gen 데커레이터 : 코루틴을 이벤트 루프에 연결하는 등 다른 서비스를 제공하는 프레임워크 중 하나. Tornado 비동기 네트워킹 라이브러리에서 제공하는 데커레이터.

yield from 은 @coroutine 데커레이터와 함께 사용할 수 없다.

## 16.5. 코루틴 종료와 예외 처리

코루틴 안에서 발생한 예외 처리x >> next()나 send()로 코루틴 호출한 호출자에 예외 전파됨...

예외를 전달할 수 있게 해주는 메서드
 generator.throw(exc_type[, exc_value[, traceback]]) : 제너레이터가 중단한 곳의 yield 표현식에 예외 전달. 예외 처리>>제어 흐름은 다음 yield문까지 진행, 생성된 값은 generator.throw() 호출 값이 됨. 예외 처리x >> 호출자 까지 예외 전파됨
 generator.close() : 제너레이터가 실행을 중단한 yield 표현식이 GeneratorExit 예외를 발생시키게 함. if 제너레이터가 예외 처리x / StopIteration 예외 >> 아무런 에러도 호출자에 전달x . GeneratorExit 예외 >> 제너레이터는 아무런 값도 생성x해야 아니면 runtimeeroor . 다른 예외는 모두 호출자에 전달 됨.
 
## 16.6. 코루틴에서 값 반환하기

averager() >> 코루틴 변형해 값 반환 ; average 값 반환/ 튜플 반환(누적된 데이터 반환) 가능

## 16.7. yield from 사용하기

gen() >> yield from subgen () 호출 >> subgen()이 값 생성 >> gen()의 호출자에 반환 *gen()은 subgen()이 종료될 때까지 실행을 중단함.

yield from x : iter(X) 호출 >> x의 반복자 가져옴
yield from 특징 : 바깥쪽 호출자와 가장 안쪽에 있는 하위 제너레이터 사이에 양방향 채널을 열어줌. >> 값 직접 주고 받음. 예외 직접 던질 수 있음 >> 코루틴 위임 가능

대표 제너레이터 ㅣ yield from <반복형> 표현식을 담고 있는 제너레이터 함수
하위 제너레이터 : yield from 표현식 중 <반복형> 에서 가져오는 제너레이터.
호출자 : 대표 제너레이터를 호출하는 코드

## 16.8. yield from의 의미

그 반복자가 또 다른 제너레이터인 경우, 하위 제너레이터의 본체가 yield from 표현식의 대상 안에 들어가는 것과 동일한 효과가 발생한다. 게다가 하위 제너레이터는 값을 가진 return 문을 이용해 값을 반환할 수 있고,  그 값은 yield from 표현식의 값이 된다.

하위 제너레이터가 생성하는 값은 모두 대표 제너레이터의 호출자에 바로 전달된다.
send()를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전다로딘다. 값이 None이면 하위 제너레이터의 __next__() 메서드가 호출된다. 전달된 값이 None이 아니면 하위 제너레이터의 send() 메서드가 호출된다. 호출된 메서드에서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
제너레이터나 하위 제너레이터에서 return expr 문을 실행하면, 제너레이터를 빠져나온 후 StopIteration(expr) 예외가 발생한다.
하위 제너레이터가 실행을 마친 후 발생한 StopIteration 예외의 첫번째 인수가 yield from 표현식의 값이 된다.

대표 제너레이터에 던져진 GeneratorExit 이외의 예외는 하위 제너레이터의 throw() 메서드에 전달된다. throw() 메서드를 호출해서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
GeneratorExit 예외가 대표 제너레이터에 던져지거나 대표 제너레이터의 close()메서드가 호출되면 하위 제너레이터의 close() 메서드가 호출된다. 그 결과 예외가 발생하면 발생한 예외가 대표 제너레이터에 전파된다. 그렇지 않으면 대표 제너레이터에서 GeneratorExit 예외가 발생한다.

_i : 하위 제너레이터
_y : 하위 제너레이터가 생성한 값
_r : 최종 결과값
_s : 호출자가 대표 제너레이터에 보낸값.하위 제너레이터에 전달됨.
_e : 예외

## 16.9. 사용 사례 : 이산 이벤트 시뮬레이션을 위한 코루틴



